# Python文件操作练习题集 📁

## 🎯 学习目标
通过这套练习题，你将掌握：
- 基础文件读写操作
- os模块的常用函数
- 路径处理和目录操作
- 文件信息获取和判断
- 实际应用场景

## 📚 必备知识点速览

### 🔧 **核心模块**
```python
import os      # 操作系统接口
import shutil  # 高级文件操作
import json    # JSON文件处理
from pathlib import Path  # 现代路径操作(Python 3.4+)
```

### 📋 **关键函数清单**
| 功能分类 | 函数 | 作用 |
|---------|------|------|
| **文件读写** | `open()`, `read()`, `write()` | 基础文件操作 |
| **路径操作** | `os.path.join()`, `os.path.exists()` | 路径拼接和检查 |
| **目录操作** | `os.listdir()`, `os.makedirs()` | 列出和创建目录 |
| **文件信息** | `os.stat()`, `os.path.getsize()` | 获取文件详细信息 |
| **文件移动** | `shutil.copy()`, `shutil.move()` | 复制和移动文件 |
| **遍历目录** | `os.walk()` | 递归遍历目录树 |

---

## 🌱 **练习等级说明**
- 🟢 **初级**: 基础操作，适合刚开始学习
- 🟡 **中级**: 需要组合多个函数
- 🔴 **高级**: 综合应用，需要逻辑思考

---


In [ ]:
# 🔧 题目9：改进版本 - 生产级临时文件管理器
import os
from pathlib import Path
from typing import List, Dict
import logging

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class TempFileManager:
    """临时文件管理器 - 使用面向对象设计，更加专业"""
    
    def __init__(self, base_directory: str = "./"):
        self.base_directory = Path(base_directory)
        self.temp_files: Dict[str, str] = {}
        logging.info(f"初始化临时文件管理器，基础目录: {self.base_directory.absolute()}")
    
    def create_temp_files(self) -> None:
        """创建测试用的临时文件"""
        temp_files = [
            "my_project/temp1.tmp",
            "my_project/src/cache.tmp", 
            "my_project/data/backup.tmp",
        ]
        
        for file_path in temp_files:
            full_path = self.base_directory / file_path
            full_path.parent.mkdir(parents=True, exist_ok=True)
            
            try:
                with open(full_path, "w", encoding='utf-8') as f:
                    f.write("这是临时文件")
                logging.info(f"创建临时文件: {full_path}")
            except OSError as e:
                logging.error(f"创建文件失败 {full_path}: {e}")
        
        print("✅ 临时文件创建完成")
    
    def find_tmp_files(self) -> Dict[str, str]:
        """查找所有.tmp文件，返回{完整路径: 文件名}字典"""
        self.temp_files.clear()
        
        if not self.base_directory.exists():
            logging.error(f"目录不存在: {self.base_directory}")
            return self.temp_files
        
        print(f"🔍 搜索目录: {self.base_directory.absolute()}")
        
        # 使用pathlib更优雅的递归查找
        for tmp_file in self.base_directory.rglob("*.tmp"):
            if tmp_file.is_file():
                self.temp_files[str(tmp_file)] = tmp_file.name
                logging.info(f"发现临时文件: {tmp_file}")
        
        self._display_temp_files()
        return self.temp_files
    
    def _display_temp_files(self) -> None:
        """显示当前找到的临时文件"""
        if not self.temp_files:
            print("📭 未找到任何.tmp文件")
            return
        
        print(f"📋 找到 {len(self.temp_files)} 个临时文件:")
        for index, (path, name) in enumerate(self.temp_files.items(), 1):
            print(f"  {index}. {name} (路径: {path})")
    
    def delete_temp_file(self, index: int) -> bool:
        """根据索引删除指定的临时文件"""
        if not self.temp_files:
            print("❌ 没有可删除的文件")
            return False
        
        if not (1 <= index <= len(self.temp_files)):
            print(f"❌ 索引无效，请输入 1-{len(self.temp_files)} 之间的数字")
            return False
        
        # 获取要删除的文件
        file_paths = list(self.temp_files.keys())
        target_path = file_paths[index - 1]
        target_name = self.temp_files[target_path]
        
        try:
            os.remove(target_path)
            print(f"✅ 已删除文件: {target_name}")
            logging.info(f"删除文件: {target_path}")
            
            # 从字典中移除已删除的文件
            del self.temp_files[target_path]
            return True
            
        except OSError as e:
            print(f"❌ 删除文件失败: {e}")
            logging.error(f"删除文件失败 {target_path}: {e}")
            return False
    
    def interactive_cleanup(self) -> None:
        """交互式清理临时文件"""
        print("\n🧹 开始交互式临时文件清理")
        
        while True:
            # 重新扫描文件（确保显示最新状态）
            self.find_tmp_files()
            
            if not self.temp_files:
                print("🎉 所有临时文件已清理完成！")
                break
            
            try:
                user_input = input(f"\n请选择要删除的文件 (1-{len(self.temp_files)}) 或输入 'exit' 退出: ").strip()
                
                if user_input.lower() == 'exit':
                    print("👋 感谢使用临时文件管理器！")
                    break
                
                index = int(user_input)
                self.delete_temp_file(index)
                
            except ValueError:
                print("❌ 请输入有效的数字或 'exit'")
            except KeyboardInterrupt:
                print("\n\n👋 程序被用户中断")
                break

# 主程序执行
if __name__ == "__main__":
    # 创建文件管理器实例
    manager = TempFileManager("./")
    
    # 执行完整的流程
    print("🚀 启动临时文件管理器")
    manager.create_temp_files()
    manager.interactive_cleanup()


# 🟢 初级练习：基础文件操作

## 题目1：文件创建与写入 📝 ✅
**任务**: 创建一个名为`my_diary.txt`的文件，写入今天的日记内容

**要求**:
1. 使用`open()`函数创建文件
2. 写入至少3行内容（可以是任意文本）
3. 正确关闭文件


In [2]:
# 题目1：在这里写你的代码
# 创建并写入日记文件

# 你的代码写在这里：
diary_content = """
今天早上6点10分起床，
起床后吃早饭，
然后开始学习Python。
"""
with open("my_diary.txt", "w", encoding="utf-8") as f:
    f.write(diary_content.strip())

# 验证文件是否创建成功
import os

if os.path.exists("my_diary.txt"):
    print("✅ 文件创建成功！")
    with open("my_diary.txt", "r", encoding="utf-8") as f:
        content = f.read()
        print("📄 文件内容：")
        print(content)
else:
    print("❌ 文件未创建，请检查代码")

✅ 文件创建成功！
📄 文件内容：
今天早上6点10分起床，
起床后吃早饭，
然后开始学习Python。


In [19]:
# 其他可能的写法（也正确，但不如你的优雅）
# 写法1：分行写入
with open("my_diary.txt", "w", encoding="utf-8") as f:
    f.write("今天早上6点10分起床，\n")
    f.write("起床后吃早饭，\n")
    f.write("然后开始学习Python。")

# 写法2：使用列表
lines = ["今天早上6点10分起床，", "起床后吃早饭，", "然后开始学习Python。"]
with open("my_diary.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

## 题目2：文件读取与处理 📖
**任务**: 读取刚才创建的`my_diary.txt`文件，统计文件中的字符数和行数

**要求**:
1. 读取文件全部内容
2. 计算总字符数（包含空格和换行符）
3. 计算总行数
4. 显示结果

**学习重点**: `read()`, `readlines()`, `len()`函数的使用


In [23]:
# 题目2：在这里写你的代码
# 读取并分析日记文件

# 你的代码写在这里：
try:
    with open("my_diary.txt", "r", encoding="utf-8") as file:
        char_num = 0
        lines = file.readlines()
        for line in lines:
            char_num += len(line)
        print(f"lines is:{lines}")
    print("📊 文件统计信息：")
    print(f"- 总字符数：{char_num} 个")
    print(f"- 总行数：{len(lines)}行")
except Exception as e:
    print("读取文件出错:", e)


# 期望输出格式：
# 📊 文件统计信息：
# - 总字符数：XX 个
# - 总行数：XX 行

lines is:['今天早上6点10分起床，\n', '起床后吃早饭，\n', '然后开始学习Python。']
📊 文件统计信息：
- 总字符数：34 个
- 总行数：3行


In [ ]:
# 更简洁的写法：
char_num = sum(len(line) for line in lines) # sum的括号中是一个生成器表达式，得到一个生成器，可迭代，所以sum可以对它们进行求和。

# 或者直接读取全部内容：
content = file.read()
char_num = len(content)

## 题目3：os模块基础 - 路径操作 🛤️ ✅
**任务**: 使用os模块完成基础路径操作

**要求**:
1. 获取当前工作目录
2. 检查`my_diary.txt`文件是否存在
3. 获取该文件的绝对路径
4. 拆分文件路径，分别获取目录和文件名

**学习重点**: `os.getcwd()`, `os.path.exists()`, `os.path.abspath()`, `os.path.split()`


In [25]:
# 题目3：在这里写你的代码
import os

# 你的代码写在这里：
current_workdir = os.getcwd()
is_my_diary_exist = os.path.exists('my_diary.txt')
abs_path = os.path.abspath('my_diary.txt')
dirname = os.path.dirname(abs_path)
filename = os.path.basename(abs_path)
print(f'📍 路径信息：')
print(f'- 当前目录：{current_workdir}')
print(f'- 文件是否存在：{is_my_diary_exist}')
print(f'- 绝对路径：{abs_path}')
print(f'- 目录部分：{dirname}')
print(f'- 文件名部分：{filename}')

# 期望输出格式：
# 📍 路径信息：
# - 当前目录：/xxx/xxx
# - 文件是否存在：True/False
# - 绝对路径：/xxx/xxx/my_diary.txt
# - 目录部分：/xxx/xxx
# - 文件名部分：my_diary.txt


📍 路径信息：
- 当前目录：/Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题
- 文件是否存在：True
- 绝对路径：/Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_diary.txt
- 目录部分：/Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题
- 文件名部分：my_diary.txt


In [26]:
# 获取目录和文件名，还可以：
dirname, filename = os.path.split(abs_path)
print(f'- 目录部分：{dirname}')
print(f'- 文件名部分：{filename}')


- 目录部分：/Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题
- 文件名部分：my_diary.txt


# 🟡 中级练习：目录操作与文件管理 ✅❌

## 题目4：创建目录结构 📁
**任务**: 创建一个项目目录结构

**要求**:
1. 创建一个名为`my_project`的主目录 ✅
2. 在主目录下创建三个子目录：`src`, `data`, `docs` ✅
3. 在`data`目录下再创建`input`和`output`两个子目录 ✅
4. 列出所有创建的目录 ❌，这个函数不太会写

**学习重点**: `os.makedirs()`, `os.listdir()`, `os.path.join()`


In [76]:
# 题目4：在这里写你的代码
import os

# 你的代码写在这里：
directory = ["src", {"data": ["input", "output"]}, "docs"]
try:
    for dir in directory:
        if isinstance(dir, dict):
            for k, v in dir.items():
                os.makedirs(f"my_project/{k}/{v[0]}")
                os.makedirs(f"my_project/{k}/{v[1]}")
        else:
            os.makedirs(f"my_project/{dir}")

except FileExistsError as e:
    print("当前的目录已存在:", e)
except Exception as e:
    print("出错:", e)

# 验证目录结构 这个方法不太会写❌
def show_directory_tree(path, prefix=""):
    """显示目录树结构"""
    if os.path.exists(path):
        print(f'{prefix}📁 {os.path.basename(path)}')
        items = os.listdir(path)
        for item in items:
            item_path = os.path.join(path,item)
            if os.path.isdir(item_path):
                show_directory_tree(item_path,prefix + "  ")

print("📁 创建的目录结构：")
show_directory_tree("my_project")
# 为什么 ./my_project在os.path.exists的时候返回True，而/my_project在os.path.exists返回False呢？

当前的目录已存在: [Errno 17] File exists: 'my_project/src'
📁 创建的目录结构：
📁 my_project
  📁 docs
  📁 data
    📁 input
    📁 output
  📁 src


### 我的答案的分析
基本正确，但是但有一些可以改进的地方

🟡 主要问题：
```python
# 你的数据结构设计过于复杂
directory = ["src", {"data": ["input", "output"]}, "docs"]
```
我的方法改进版
```python
import os

try:
    # 创建主目录
    os.makedirs("my_project", exist_ok=True)
    
    # 创建子目录
    subdirs = [
        "src",
        "docs", 
        "data/input",
        "data/output"
    ]
    
    for subdir in subdirs:
        full_path = os.path.join("my_project", subdir)
        os.makedirs(full_path, exist_ok=True)
        print(f"✅ 创建目录: {full_path}")
        
except Exception as e:
    print(f"❌ 创建目录失败: {e}")
```


### 🤔疑问
1. show_directory_tree中的prefix的作用是什么？

prefix参数用于控制输出的缩进层级，让目录树有视觉层次感：
```python
def show_directory_tree(path, prefix=""):
    print(f'{prefix}📁 {path}')
    # ...
    show_directory_tree(item_path, prefix + "  ")  # 递归时增加缩进
```
#### 有prefix缩进（清晰）✅


#### 无prefix缩进（混乱）❌

2. 为什么要输出os.path.basename(path)，而不是直接输出path?

原因：避免重复的路径信息

#### 不用 basename 的输出（有重复路径信息）

#### 使用 basename 的输出（更简洁）✅  

3. 为什么 ./my_project在os.path.exists的时候返回True，而/my_project在os.path.exists返回False呢？

`. `代表当前目录，所以` ./my_project `是从当前工作目录开始查找。而 `/my_project` 代表从根目录开始查找，这显然不存在。这就像网页URL中，相对路径和绝对路径的区别。

## 题目5：文件信息获取 📊 ✅❌
**任务**: 获取文件的详细信息

**要求**:
1. 获取`my_diary.txt`文件的大小（字节数）✅
2. 获取文件的修改时间 ❌
3. 判断它是文件还是目录 ✅
4. 获取文件的完整统计信息（使用os.stat()）✅

**学习重点**: `os.path.getsize()`, `os.path.getmtime()`, `os.path.isfile()`, `os.stat()`


In [96]:
# 题目5：在这里写你的代码
import os
from datetime import datetime

# 你的代码写在这里：
diary_path = os.path.abspath('my_diary.txt')
print('diary_path:',diary_path)
print(str(os.path.getsize(diary_path)))
print('📊 文件信息：')
print(f'- 文件大小：{os.path.getsize(diary_path)}字节')
print(f'- 修改时间：{datetime.strptime('%Y-%m-%d %H:%m:%S',str(os.path.getmtime(diary_path)))}')
print(f'- 类型：{'文件' if os.path.isfile(diary_path) else '其他文件'}')
print(f'- 详细统计信息：{os.stat(diary_path)}')

# 期望输出格式：
# 📊 文件信息：
# - 文件大小：XX 字节
# - 修改时间：2024-01-15 10:30:45
# - 类型：文件
# - 详细统计信息：... 


diary_path: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_diary.txt
80
📊 文件信息：
- 文件大小：80字节
- 修改时间：2025-06-09 11:03:29
- 类型：文件
- 详细统计信息：os.stat_result(st_mode=33188, st_ino=24537465, st_dev=16777230, st_nlink=1, st_uid=502, st_gid=20, st_size=80, st_atime=1749450376, st_mtime=1749438209, st_ctime=1749438209)


## 🎯 我的答案部分正确，但有一个重要错误需要修正
### ❌ 主要错误：修改时间处理
```python
# 你的代码（错误）❌
datetime.strptime('%Y-%m-%d %H:%m:%S', str(os.path.getmtime(diary_path)))
```
错误分析：
1. 参数顺序错误：strptime(格式, 时间字符串) → 应该是 strptime(时间字符串, 格式)
2. 数据类型错误：os.path.getmtime()返回时间戳(浮点数)，不是格式化字符串
3. 格式字符串错误：%m是月份，应该用%M表示分钟
4. 应该用fromtimestamp()：时间戳转换用fromtimestamp()而不是strptime()
### 🔧 正确的修改时间处理：
```python
# 方法1：分步处理（推荐，便于理解）
mtime_timestamp = os.path.getmtime(diary_path)  # 获取时间戳
mtime_datetime = datetime.fromtimestamp(mtime_timestamp)  # 转换为datetime对象
formatted_time = mtime_datetime.strftime('%Y-%m-%d %H:%M:%S')  # 格式化

# 方法2：一行代码
formatted_time = datetime.fromtimestamp(os.path.getmtime(diary_path)).strftime('%Y-%m-%d %H:%M:%S')
```
### 💡 知识点解析：
**时间戳 vs 格式化时间**

```python
# 时间戳（浮点数）
timestamp = os.path.getmtime('file.txt')  # 例如：1705294245.123456

# 转换为可读时间
readable_time = datetime.fromtimestamp(timestamp)  # datetime对象
formatted = readable_time.strftime('%Y-%m-%d %H:%M:%S')  # 字符串
```
**strptime vs fromtimestamp**
```python
# strptime：解析时间字符串
datetime.strptime('2024-01-15 10:30:45', '%Y-%m-%d %H:%M:%S')

# fromtimestamp：时间戳转datetime
datetime.fromtimestamp(1705294245.123456)
```

## 题目6：文件复制与移动 🚚 ✅，但是可以更好
**任务**: 使用shutil模块进行文件操作

**要求**:
1. 将`my_diary.txt`复制到`my_project/docs/`目录下，重命名为`diary_backup.txt`
2. 创建一个新文件`temp.txt`在当前目录
3. 将`temp.txt`移动到`my_project/src/`目录下

**学习重点**: `shutil.copy()`, `shutil.move()`, 文件路径组合


In [98]:
# 题目6：在这里写你的代码
import os
import shutil

# 你的代码写在这里：
try:
    diary_path = os.path.abspath('my_diary.txt')
    diary_backup = shutil.copy2(diary_path,'my_project/docs/diary_backup.txt') 
    current_path = os.getcwd()
    with open(f'{current_path}/temp.txt','w',encoding='utf-8') as f:
        pass   
    shutil.move('temp.txt','my_project/src/')
except FileNotFoundError as e:
    print('没找到')
except Exception as e:
    print('出错了')

# 验证操作结果
print("🔍 验证文件操作结果：")
if os.path.exists("my_project/docs/diary_backup.txt"):
    print("✅ diary_backup.txt 复制成功")
else:
    print("❌ diary_backup.txt 复制失败")

if os.path.exists("my_project/src/temp.txt"):
    print("✅ temp.txt 移动成功")
else:
    print("❌ temp.txt 移动失败")


🔍 验证文件操作结果：
✅ diary_backup.txt 复制成功
✅ temp.txt 移动成功


## 🎯 我的答案分析

功能都可以完成，但是有需要改进的地方。

简化文件创建

```python
# 你的方法（正确但略复杂）
current_path = os.getcwd()
with open(f'{current_path}/temp.txt', 'w', encoding='utf-8') as f:
    pass

# 更简洁的方法 ✅
with open('temp.txt', 'w', encoding='utf-8') as f:
    f.write('这是临时文件')  # 或者 pass 也可以

# 最简洁的方法
open('temp.txt', 'w').close()  # 创建空文件并立即关闭
```

## 🔧 完整的优化版本：

```python
import os
import shutil

print("🚚 开始文件操作...")

try:
    # 1. 复制日记文件
    source_file = 'my_diary.txt'
    target_file = 'my_project/docs/diary_backup.txt'

    if os.path.exists(source_file):
        shutil.copy2(source_file, target_file)
        print("✅ my_diary.txt 复制成功")
    else:
        print("❌ my_diary.txt 不存在")

    # 2. 创建临时文件
    with open('temp.txt', 'w', encoding='utf-8') as f:
        f.write('这是一个临时文件')
    print("✅ temp.txt 创建成功")

    # 3. 移动临时文件
    shutil.move('temp.txt', 'my_project/src/')
    print("✅ temp.txt 移动成功")

except FileNotFoundError as e:
    print(f"❌ 文件未找到: {e}")
except PermissionError as e:
    print(f"❌ 权限不足: {e}")
except Exception as e:
    print(f"❌ 操作失败: {e}")

print("\n🔍 验证操作结果：")
# ... 验证代码保持不变
```

## 🤔 关于你代码中的细节问题：
**Q: 为什么要获取当前路径再拼接？**
```python
current_path = os.getcwd()
with open(f'{current_path}/temp.txt', 'w') as f:
```
A: 其实不需要这么做，因为：
- 直接用 'temp.txt' 就会在当前目录创建
- os.getcwd() + 拼接 = 多余的操作

# 🔴 高级练习：综合应用

## 题目7：目录遍历与统计 🔍
**任务**: 创建一个目录分析器

**要求**:
1. 使用`os.walk()`递归遍历`my_project`目录
2. 统计总文件数和总目录数
3. 列出所有文件的路径和大小
4. 计算所有文件的总大小

**学习重点**: `os.walk()`的使用，递归遍历的概念


In [128]:
# 题目7：在这里写你的代码
import os

# ❌错误的答案
def analyze_directory(path):
    """分析目录的函数"""
    # 你的代码写在这里：
    total_files = 0
    total_dirs = 0
    total_file_size = 0
    file_list = []
    try:
        # 错误❌答案
        # if os.path.exists(path):
        #     directory = os.walk(path)
        #     for dir in directory:
        #         print("dir:", dir)
        #         for item in dir:
        #             print("item:", item)
        #             if not isinstance(item, list):
        #                 if os.path.isdir(item):
        #                     total_dirs += 1
        #             else:
        #                 if len(item) > 0:
        #                     print('进入这里了')
        #                     for i in item:
        #                         print('i:',i,'os.path.isfile(i):',os.path.isfile(os.path.abspath(i)),'os.path.abspath(i):',os.path.abspath(i))
        #                         if os.path.isfile((os.path.abspath(i))):
        #                             total_files += 1
        #                             total_file_size += os.path.getsize(os.path.abspath(i))
        # 正确✅答案
        if os.path.exists(path):
            for root,dirs,filename in os.walk(path):
                print('root:',root)
                print('dirs:',dirs)
                print('filename:',filename)
                print('='*50)
                total_dirs += len(dirs)
                total_files += len(filename)
                for file in filename:
                    file_path = os.path.join(root,file)
                    total_file_size += os.path.getsize(file_path)
                    file_list.append(file_path)
            

            print("📊 目录分析结果：")
            print(f"- 总目录数：{total_dirs} 个")
            print(f"- 总文件数：{total_files} 个")
            print(f"- 文件列表：")
            for file in file_list:
                print(f'    📄 {file} ({os.path.getsize(file)})字节')
            print("")
            print(f"- 总文件大小：{total_file_size} 字节")
    except FileExistsError as e:
        print("文件不存在:", e)
    except Exception as e:
        print("分析文件出错:", e)


# 调用函数分析my_project目录
if os.path.exists("my_project"):
    analyze_directory("my_project")
else:
    print("❌ my_project目录不存在，请先完成前面的题目")

# 期望输出格式：
# 📊 目录分析结果：
# - 总目录数：X 个
# - 总文件数：X 个
# - 文件列表：
#   📄 my_project/docs/diary_backup.txt (XX 字节)
#   📄 my_project/src/temp.txt (XX 字节)
# - 总文件大小：XX 字节

root: my_project
dirs: ['docs', 'data', 'src']
filename: []
root: my_project/docs
dirs: []
filename: ['diary_backup.txt']
root: my_project/data
dirs: ['input', 'output']
filename: []
root: my_project/data/input
dirs: []
filename: []
root: my_project/data/output
dirs: []
filename: []
root: my_project/src
dirs: []
filename: ['temp.txt']
📊 目录分析结果：
- 总目录数：5 个
- 总文件数：2 个
- 文件列表：
    📄 my_project/docs/diary_backup.txt (80)字节
    📄 my_project/src/temp.txt (0)字节

- 总文件大小：80 字节


## 📝 **题目7代码分析：你的思路正确，但对`os.walk()`理解有误**

### ❌ **主要问题分析**

#### **1. 对`os.walk()`返回值的误解**
```python
# 你的理解（错误）
for dir in directory:  # dir实际是三元组：(root, dirs, files)
    for item in dir:   # item是三元组中的一个元素
```

**正确理解：**
`os.walk(path)`每次迭代返回一个**三元组**：`(当前目录路径, 子目录列表, 文件列表)`

```python
# 正确的解构方式
for root, dirs, files in os.walk(path):
    # root: 当前目录路径（字符串）
    # dirs: 子目录名列表 
    # files: 文件名列表
```

#### **2. 路径处理错误**
```python
# 你的代码（错误）
os.path.abspath(i)  # i只是文件名，没有完整路径
```

**正确方式：**
```python
file_path = os.path.join(root, filename)  # 拼接完整路径
```

#### **3. 目录计数逻辑混乱**
你的代码试图通过`isinstance(item, list)`和`os.path.isdir()`来判断，但逻辑很复杂且容易出错。

---

### ✅ **正确的实现方式**


In [ ]:
# 📚 标准答案：正确的目录分析实现

def analyze_directory_correct(path):
    """正确的目录分析函数"""
    total_files = 0
    total_dirs = 0 
    total_file_size = 0
    file_list = []
    
    try:
        if not os.path.exists(path):
            print(f"❌ 目录不存在：{path}")
            return
            
        print(f"🔍 开始分析目录：{path}")
        
        # 正确使用os.walk()
        for root, dirs, files in os.walk(path):
            print(f"📂 当前目录：{root}")
            print(f"   子目录：{dirs}")
            print(f"   文件：{files}")
            
            # 统计子目录数（不包含当前目录本身）
            if root != path:  # 排除根目录本身
                total_dirs += 1
            
            # 统计文件
            for filename in files:
                file_path = os.path.join(root, filename)  # 🔑 关键：拼接完整路径
                
                if os.path.isfile(file_path):  # 确认是文件
                    file_size = os.path.getsize(file_path)
                    total_files += 1
                    total_file_size += file_size
                    
                    # 记录文件信息
                    file_list.append({
                        'path': file_path,
                        'size': file_size
                    })
        
        # 输出结果
        print(f"\n📊 目录分析结果：")
        print(f"- 总目录数：{total_dirs} 个")
        print(f"- 总文件数：{total_files} 个")
        print(f"- 文件列表：")
        
        for file_info in file_list:
            print(f"  📄 {file_info['path']} ({file_info['size']} 字节)")
            
        print(f"- 总文件大小：{total_file_size} 字节")
        
    except Exception as e:
        print(f"❌ 分析过程出错：{e}")

# 运行正确版本
print("=" * 60)
print("🎯 正确版本的目录分析")
print("=" * 60)

if os.path.exists("my_project"):
    analyze_directory_correct("my_project")
else:
    print("❌ my_project目录不存在")


### 🔍 **关键知识点详解**

#### **1. `os.walk()`的工作原理**
```python
# os.walk()每次迭代返回三个值
for root, dirs, files in os.walk("my_project"):
    # root = "my_project"          -> 当前目录路径
    # dirs = ["src", "docs", "data"] -> 子目录名列表
    # files = []                   -> 文件名列表
```

**可视化理解：**
```
my_project/                 <- root="my_project", dirs=["src","docs","data"], files=[]
├── src/                    <- root="my_project/src", dirs=[], files=["temp.txt"]
│   └── temp.txt
├── docs/                   <- root="my_project/docs", dirs=[], files=["diary_backup.txt"]  
│   └── diary_backup.txt
└── data/                   <- root="my_project/data", dirs=["input","output"], files=[]
    ├── input/              <- root="my_project/data/input", dirs=[], files=[]
    └── output/             <- root="my_project/data/output", dirs=[], files=[]
```

#### **2. 路径拼接的重要性**
```python
# 错误方式（你的代码）
filename = "temp.txt"
os.path.abspath(filename)  # 得到：/当前工作目录/temp.txt （错误路径）

# 正确方式
root = "my_project/src"
filename = "temp.txt"  
file_path = os.path.join(root, filename)  # 得到：my_project/src/temp.txt
```

#### **3. 为什么你的统计结果不对？**
- **总目录数：6个** ✅ 这个是对的
- **总文件数：0个** ❌ 错误！应该是2个

**原因：** 你用`os.path.abspath(i)`时，`i`只是文件名（如"temp.txt"），不包含它所在的目录路径，所以`os.path.isfile()`返回`False`。

---

### 💡 **学习要点总结**

1. **理解数据结构**：`os.walk()`返回三元组，要正确解构
2. **路径处理**：文件名需要和目录路径拼接才是完整路径
3. **逻辑清晰**：分别处理目录和文件，避免复杂的类型判断
4. **调试技巧**：你的调试输出很有用，帮助理解数据结构

### 🎯 **修改建议**

把你的函数改成这样就对了：
```python
def analyze_directory(path):
    total_files = 0
    total_dirs = 0
    total_file_size = 0
    
    for root, dirs, files in os.walk(path):  # 正确解构
        total_dirs += len(dirs)  # 直接计算子目录数，我也是这样做的，是可以的✅，答案是排除掉根目录，两种办法都可以。
        
        for filename in files:  # 遍历文件名
            file_path = os.path.join(root, filename)  # 拼接完整路径
            total_files += 1
            total_file_size += os.path.getsize(file_path)
```

你的思路是对的，只是对`os.walk()`的理解需要调整一下！ 🚀


## 题目8：配置文件处理 ⚙️ ✅错，大部分是✅的，有细节需要改进
**任务**: 创建和读取JSON配置文件

**要求**:
1. 创建一个配置字典，包含应用名称、版本、作者等信息
2. 将配置保存为`my_project/config.json`文件
3. 读取配置文件并显示所有配置项
4. 修改其中一个配置项并保存

**学习重点**: JSON文件的读写，字典操作


In [137]:
# 题目8：在这里写你的代码
import json
import os

# 你的代码写在这里：
config = {"app_name": "My Application", "version": "1.0.0", "author": "Your Name"}
json_path = f'my_project/config.json'
def update_json(config_file):
    with open(json_path,'w',encoding='utf-8') as file:
        json.dump(config_file,file,ensure_ascii=False,indent=2)
    print('✅ 配置更新成功')

try:
    print('⚙️ 配置文件操作：')
    with open(json_path,'w',encoding='utf-8') as file:
        json.dump(config,file,ensure_ascii=False,indent=2)
    print('✅ 配置文件创建成功')
    # 读取配置文件
    with open(json_path,'r',encoding='utf-8') as file:
        config_dict = json.load(file)
        # print('📄 当前配置：')
        # print('{') # 手动格式化不可取
        # print(f'    "app_name":"{config_dict['app_name']}"')
        # print(f'    "version":"{config_dict['version']}"')
        # print(f'    "author":"{config_dict['author']}"')
        # print('}')
        print(json.dumps(config_dict,ensure_ascii=False,indent=2)) # 自动格式化
    config['app_name'] = '黄文鑫'
    update_json(config)
except FileNotFoundError as e:
    print("文件没有找到:",e)
except Exception as e:
    print('写入文件出错:',e)

# 期望输出格式：
# ⚙️ 配置文件操作：
# ✅ 配置文件创建成功
# 📄 当前配置：
# {
#   "app_name": "My Application",
#   "version": "1.0.0",
#   "author": "Your Name"
# }
# ✅ 配置更新成功

⚙️ 配置文件操作：
✅ 配置文件创建成功
{
  "app_name": "My Application",
  "version": "1.0.0",
  "author": "Your Name"
}
✅ 配置更新成功


## 📋 题目8代码分析与改进建议

### ✅ 你做得好的地方：
1. **功能完整性**：成功实现了创建、读取、修改配置文件的完整流程
2. **异常处理**：使用了try-except处理潜在的文件操作错误
3. **代码组织**：将更新逻辑封装成函数，体现了良好的代码组织意识
4. **编码处理**：正确使用了UTF-8编码，确保中文内容正常显示

### ⚠️ 需要改进的问题：

#### 1. **f-string使用不当**
```python
# 当前代码：
json_path = f'my_project/config.json'  # 没有变量插值，不需要f-string

# 改进：
json_path = 'my_project/config.json'
```

#### 2. **函数设计不合理**
```python
# 当前代码：
def update_json(config_file):  # 参数名误导，实际传入的是config字典
    with open(json_path,'w',encoding='utf-8') as file:  # 使用全局变量

# 改进：
def update_json(config_dict, file_path):
    """更新JSON配置文件"""
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(config_dict, file, ensure_ascii=False, indent=2)
    print('✅ 配置更新成功')
```

#### 3. **手动构造JSON格式字符串**
```python
# 当前代码：
print('{')
print(f'    "app_name":"{config_dict['app_name']}"')
print(f'    "version":"{config_dict['version']}"')
print(f'    "author":"{config_dict['author']}"')
print('}')

# 改进：
print(json.dumps(config_dict, ensure_ascii=False, indent=2))
```

#### 4. **缺少最终状态验证**
修改配置后应该重新读取并显示更新后的内容。

### 🔧 完整的改进版本：

```python
import json
import os

def create_config_file(config_dict, file_path):
    """创建配置文件"""
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(config_dict, file, ensure_ascii=False, indent=2)
    print('✅ 配置文件创建成功')

def read_config_file(file_path):
    """读取配置文件"""
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

def update_config_file(config_dict, file_path):
    """更新配置文件"""
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(config_dict, file, ensure_ascii=False, indent=2)
    print('✅ 配置更新成功')

def display_config(config_dict, title="当前配置"):
    """显示配置内容"""
    print(f'📄 {title}：')
    print(json.dumps(config_dict, ensure_ascii=False, indent=2))

# 主逻辑
config = {"app_name": "My Application", "version": "1.0.0", "author": "Your Name"}
json_path = 'my_project/config.json'

try:
    print('⚙️ 配置文件操作：')
    
    # 1. 创建配置文件
    create_config_file(config, json_path)
    
    # 2. 读取并显示配置
    current_config = read_config_file(json_path)
    display_config(current_config)
    
    # 3. 修改配置
    current_config['app_name'] = '黄文鑫'
    update_config_file(current_config, json_path)
    
    # 4. 验证修改结果
    updated_config = read_config_file(json_path)
    display_config(updated_config, "更新后配置")
    
except FileNotFoundError as e:
    print(f"文件未找到: {e}")
except json.JSONDecodeError as e:
    print(f"JSON格式错误: {e}")
except Exception as e:
    print(f'操作失败: {e}')
```

### 📈 改进要点总结：
1. **函数职责单一**：每个函数只做一件事
2. **参数传递明确**：避免使用全局变量
3. **代码可重用**：函数可以在其他地方复用
4. **错误处理具体**：针对不同错误类型给出具体提示
5. **输出格式统一**：使用json.dumps()保持格式一致性

### 🎯 学习收获：
你的代码已经很接近正确答案了！主要是一些细节优化和最佳实践的问题。继续保持这种学习态度！

## 题目9：文件清理工具 🧹 ✅❌ 60/100分，存在问题
**任务**: 创建一个文件清理功能

**要求**:
1. 创建几个临时文件（扩展名为.tmp）在各个目录中
2. 编写函数找出所有的.tmp文件
3. 询问用户是否删除这些文件
4. 删除用户确认的文件

**学习重点**: 文件过滤，用户交互，文件删除


In [178]:
# 题目9：在这里写你的代码
import os

file_dict = {}


def create_temp_files():
    """创建一些临时文件用于测试"""
    temp_files = [
        "my_project/temp1.tmp",
        "my_project/src/cache.tmp",
        "my_project/data/backup.tmp",
    ]

    for file_path in temp_files:
        with open(file_path, "w") as f:
            f.write("这是临时文件")

    print("✅ 临时文件创建完成")


def output_current_file_dict():
    for index, (path, name) in enumerate(file_dict.items()):
        print(f"{index+1}. 当前的tmp文件的路径是: {path}，文件名是: {name}")


def find_tmp_files(directory):
    """找出所有.tmp文件"""
    # 你的代码写在这里：
    global file_dict
    if os.path.exists(directory):
        print(f"当前搜索的路径是:{directory}")
        for root, dirs, files in os.walk(directory):
            for file in files:
                if ".tmp" in file:
                    file_path = os.path.join(root, file)
                    file_dict[file_path] = file
        output_current_file_dict()
    else:
        print("当前目录不存在")


def clean_tmp_files(directory, user_choose):
    """清理.tmp文件"""
    # 你的代码写在这里：
    global file_dict  # ❌ 声明这个变量是全局变量，如果不声明，会报错UnboundLocalError: cannot access local variable 'file_dict' where it is not associated with a value，因为我在后面：file_dict = dict(file_list)，这会把全局变量覆盖掉，这是因为Python会将函数内的赋值操作视为创建局部变量，而不是修改全局变量。
    if os.path.exists(directory):
        print(f"当前需要删除tmp文件的目录是:", directory)
        find_tmp_files(directory)
        file_list = list(file_dict.items())
        for i in range(len(file_list)):
            print("i:", i, "对应的值:", file_list[i])
            if user_choose == (i + 1):
                print(f"当前需要删除的文件是: {file_list[i]} 下的 {file_list[i][1]}")
                os.remove(file_list[i][0])
        file_dict = dict(file_list)
        print("删除后，输出一次file_dict的值:")
        output_current_file_dict()
    else:
        print("当前目录不存在")


if __name__ == "__main__":
    # 执行清理
    create_temp_files()
    print("\n🔍 开始查找临时文件...")
    find_tmp_files("./")
    while True:
        if len(file_dict) <= 0:
            print("len(file_dict):", len(file_dict))
            break
        user_choose = input(
            f"请输入你要删除的文件,在1-{len(file_dict)}之间,输入exit退出:"
        )
        if user_choose == "exit":
            print("感谢你的使用🙏")
            break
        num = int(user_choose)
        if num < 1 or num > 11:
            print("请输入正确的文件索引!")
            continue
        clean_tmp_files("./", num)

✅ 临时文件创建完成

🔍 开始查找临时文件...
当前搜索的路径是:./
1. 当前的tmp文件的路径是: ./backup_files/backup.tmp，文件名是: backup.tmp
2. 当前的tmp文件的路径是: ./backup_files/subfolder/sb.tmp，文件名是: sb.tmp
3. 当前的tmp文件的路径是: ./my_project/temp1.tmp，文件名是: temp1.tmp
4. 当前的tmp文件的路径是: ./my_project/mp.tmp，文件名是: mp.tmp
5. 当前的tmp文件的路径是: ./my_project/data/data.tmp，文件名是: data.tmp
6. 当前的tmp文件的路径是: ./my_project/data/backup.tmp，文件名是: backup.tmp
7. 当前的tmp文件的路径是: ./my_project/data/input/input.tmp，文件名是: input.tmp
8. 当前的tmp文件的路径是: ./my_project/src/cache.tmp，文件名是: cache.tmp
感谢你的使用🙏


## 📋 题目9代码分析与改进建议

### ✅ 你做得好的地方：
1. **功能完整性**：成功实现了创建、查找、删除临时文件的完整流程
2. **交互设计**：提供了用户友好的交互界面，支持逐个删除文件
3. **文件操作**：正确使用了`os.walk()`遍历目录结构
4. **错误预防**：添加了目录存在性检查

### ⚠️ 存在的关键问题：

#### 1. **删除逻辑严重Bug** 🐛
```python
# 当前代码问题：
os.remove(file_list[i][0])  # 删除文件
file_dict = dict(file_list)  # ❌ 重新用原列表构建字典，删除的文件仍在字典中！
```

#### 2. **用户输入验证错误**
```python
# 当前代码：
if num < 1 or num > 11:  # ❌ 硬编码11，应该是len(file_dict)
```

#### 3. **全局变量使用不当**
全局变量增加了代码复杂性和维护难度。

#### 4. **无错误处理**
文件删除操作没有异常处理，可能导致程序崩溃。

In [185]:
# 🔧 题目9：改进版本 - 使用纯os模块的生产级临时文件管理器
import os
import logging
from typing import Dict

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class TempFileManager:
    """临时文件管理器 - 使用面向对象设计，纯os模块实现"""
    
    def __init__(self, base_directory: str = "./"):
        self.base_directory = os.path.abspath(base_directory)
        self.temp_files: Dict[str, str] = {}
        logging.info(f"初始化临时文件管理器，基础目录: {self.base_directory}")
    
    def create_temp_files(self) -> None:
        """创建测试用的临时文件"""
        temp_files = [
            "my_project/temp1.tmp",
            "my_project/src/cache.tmp", 
            "my_project/data/backup.tmp",
        ]
        
        for file_path in temp_files:
            full_path = os.path.join(self.base_directory, file_path)
            directory = os.path.dirname(full_path)
            
            # 创建目录（如果不存在）
            if not os.path.exists(directory):
                try:
                    os.makedirs(directory, exist_ok=True)
                except OSError as e:
                    logging.error(f"创建目录失败 {directory}: {e}")
                    continue
            
            # 创建文件
            try:
                with open(full_path, "w", encoding='utf-8') as f:
                    f.write("这是临时文件")
                logging.info(f"创建临时文件: {full_path}")
            except OSError as e:
                logging.error(f"创建文件失败 {full_path}: {e}")
        
        print("✅ 临时文件创建完成")
    
    def find_tmp_files(self) -> Dict[str, str]:
        """查找所有.tmp文件，返回{完整路径: 文件名}字典"""
        self.temp_files.clear()
        
        if not os.path.exists(self.base_directory):
            logging.error(f"目录不存在: {self.base_directory}")
            return self.temp_files
        
        print(f"🔍 搜索目录: {self.base_directory}")
        
        # 使用os.walk()递归查找所有.tmp文件
        for root, dirs, files in os.walk(self.base_directory):
            for filename in files:
                if filename.endswith('.tmp'):  # ✅ 精确匹配.tmp文件
                    file_path = os.path.join(root, filename)
                    self.temp_files[file_path] = filename
                    logging.info(f"发现临时文件: {file_path}")
        
        self._display_temp_files()
        return self.temp_files
    
    def _display_temp_files(self) -> None:
        """显示当前找到的临时文件
        这个函数是用下划线开头的，有什么特殊含义吗？
            在Python中，以单下划线_开头的方法名表示这是一个"内部使用"的方法，是一种命名约定。
            它暗示其他开发者这个方法主要供类内部使用，不建议在类外部直接调用。这不是强制性的限制，只是一种编程惯例（与双下划线__不同）。
            
        怎么样叫做在类的内部使用，怎么样叫做在外部使用？
            内部使用：在类的其他方法中调用该方法，如当前代码中find_tmp_files()方法调用self._display_temp_files()。
            外部使用：创建类的实例后直接调用该方法，如temp_manager._display_temp_files()。
        """
        if not self.temp_files:
            print("📭 未找到任何.tmp文件")
            return
        
        print(f"📋 找到 {len(self.temp_files)} 个临时文件:")
        for index, (path, name) in enumerate(self.temp_files.items(), 1):
            print(f"  {index}. {name} (路径: {path})")
    
    def delete_temp_file(self, index: int) -> bool:
        """根据索引删除指定的临时文件"""
        if not self.temp_files:
            print("❌ 没有可删除的文件")
            return False
        
        if not (1 <= index <= len(self.temp_files)):
            print(f"❌ 索引无效，请输入 1-{len(self.temp_files)} 之间的数字")
            return False
        
        # 获取要删除的文件
        file_paths = list(self.temp_files.keys())
        target_path = file_paths[index - 1]
        target_name = self.temp_files[target_path]
        
        try:
            os.remove(target_path)
            print(f"✅ 已删除文件: {target_name}")
            logging.info(f"删除文件: {target_path}")
            
            # 🔑 关键修复：从字典中移除已删除的文件
            del self.temp_files[target_path]
            return True
            
        except OSError as e:
            print(f"❌ 删除文件失败: {e}")
            logging.error(f"删除文件失败 {target_path}: {e}")
            return False
    
    def interactive_cleanup(self) -> None:
        """交互式清理临时文件"""
        print("\n🧹 开始交互式临时文件清理")
        
        while True:
            # 重新扫描文件（确保显示最新状态）
            self.find_tmp_files()
            
            if not self.temp_files:
                print("🎉 所有临时文件已清理完成！")
                break
            
            try:
                user_input = input(f"\n请选择要删除的文件 (1-{len(self.temp_files)}) 或输入 'exit' 退出: ").strip()
                
                if user_input.lower() == 'exit':
                    print("👋 感谢使用临时文件管理器！")
                    break
                
                index = int(user_input)
                self.delete_temp_file(index)
                
            except ValueError:
                print("❌ 请输入有效的数字或 'exit'")
            except KeyboardInterrupt:
                print("\n\n👋 程序被用户中断")
                break

# 函数式版本（如果你更喜欢函数而不是类）
def create_temp_files_simple():
    """简单版本：创建临时文件"""
    temp_files = [
        "my_project/temp1.tmp",
        "my_project/src/cache.tmp", 
        "my_project/data/backup.tmp",
    ]
    
    for file_path in temp_files:
        directory = os.path.dirname(file_path)
        if not os.path.exists(directory):
            os.makedirs(directory, exist_ok=True)
        
        with open(file_path, "w", encoding='utf-8') as f:
            f.write("这是临时文件")
    
    print("✅ 临时文件创建完成")

def find_tmp_files_simple(directory="./"):
    """简单版本：查找.tmp文件"""
    tmp_files = {}
    
    if not os.path.exists(directory):
        print(f"❌ 目录不存在: {directory}")
        return tmp_files
    
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith('.tmp'):
                file_path = os.path.join(root, filename)
                tmp_files[file_path] = filename
    
    return tmp_files

def display_files_simple(tmp_files):
    """简单版本：显示文件列表"""
    if not tmp_files:
        print("📭 未找到任何.tmp文件")
        return
    
    print(f"📋 找到 {len(tmp_files)} 个临时文件:")
    for index, (path, name) in enumerate(tmp_files.items(), 1):
        print(f"  {index}. {name} (路径: {path})")

def delete_file_simple(file_path, file_name):
    """简单版本：删除单个文件"""
    try:
        os.remove(file_path)
        print(f"✅ 已删除文件: {file_name}")
        return True
    except OSError as e:
        print(f"❌ 删除文件失败: {e}")
        return False

# 主程序执行
if __name__ == "__main__":
    print("选择执行方式：")
    print("1. 使用类版本（推荐）")
    print("2. 使用函数版本（简单）")
    
    choice = input("请输入选择 (1 或 2): ").strip()
    
    if choice == "1":
        # 🚀 类版本 - 功能完整
        print("\n" + "="*50)
        print("🚀 启动临时文件管理器（类版本）")
        print("="*50)
        
        manager = TempFileManager("./")
        manager.create_temp_files()
        manager.interactive_cleanup()
        
    elif choice == "2":
        # 📝 函数版本 - 简单直接
        """
        你可能会发现，这里的print会慢于create_temp_files_simple中的logging的输出，是因为：
        print 和 logging 使用了不同的输出流。
        logging 默认使用 stderr（标准错误流），而 print 使用 stdout（标准输出流）。
        stderr 是无缓冲的，会立即输出，而 stdout 是行缓冲的，可能会稍有延迟。这就导致了 logging 的输出会先于 print 显示在控制台中。
        """
        print("\n" + "="*50) 
        print("📝 启动临时文件管理器（函数版本）")
        print("="*50)
        
        # 创建临时文件
        create_temp_files_simple()
        
        # 主循环
        while True:
            # 查找文件
            tmp_files = find_tmp_files_simple("./")
            display_files_simple(tmp_files)
            
            if not tmp_files:
                print("🎉 没有找到临时文件！")
                break
                
            # 用户交互
            try:
                user_input = input(f"\n请选择要删除的文件 (1-{len(tmp_files)}) 或输入 'exit' 退出: ").strip()
                
                if user_input.lower() == 'exit':
                    print("👋 感谢使用临时文件管理器！")
                    break
                
                index = int(user_input)
                if 1 <= index <= len(tmp_files):
                    file_list = list(tmp_files.items())
                    file_path, file_name = file_list[index - 1]
                    delete_file_simple(file_path, file_name)
                else:
                    print(f"❌ 请输入 1-{len(tmp_files)} 之间的数字")
                    
            except ValueError:
                print("❌ 请输入有效的数字或 'exit'")
            except KeyboardInterrupt:
                print("\n\n👋 程序被用户中断")
                break
    
    else:
        print("❌ 无效选择，请重新运行程序")

选择执行方式：
1. 使用类版本（推荐）
2. 使用函数版本（简单）


2025-06-09 20:21:04,365 - INFO - 初始化临时文件管理器，基础目录: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题
2025-06-09 20:21:04,366 - INFO - 创建临时文件: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_project/temp1.tmp
2025-06-09 20:21:04,367 - INFO - 创建临时文件: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_project/src/cache.tmp
2025-06-09 20:21:04,368 - INFO - 创建临时文件: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_project/data/backup.tmp
2025-06-09 20:21:04,369 - INFO - 发现临时文件: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/backup_files/backup.tmp
2025-06-09 20:21:04,369 - INFO - 发现临时文件: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/backup_files/subfolder/sb.tmp
2025-06-09 20:21:04,369 - INFO - 发现临时文件: /Users/huangwenxin/scripts/Code/python-learning-process


🚀 启动临时文件管理器（类版本）
✅ 临时文件创建完成

🧹 开始交互式临时文件清理
🔍 搜索目录: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题
📋 找到 6 个临时文件:
  1. backup.tmp (路径: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/backup_files/backup.tmp)
  2. sb.tmp (路径: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/backup_files/subfolder/sb.tmp)
  3. temp1.tmp (路径: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_project/temp1.tmp)
  4. mp.tmp (路径: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_project/mp.tmp)
  5. backup.tmp (路径: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_project/data/backup.tmp)
  6. cache.tmp (路径: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_project/src/cache.tmp)


2025-06-09 20:21:12,944 - INFO - 删除文件: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_project/src/cache.tmp
2025-06-09 20:21:12,946 - INFO - 发现临时文件: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/backup_files/backup.tmp
2025-06-09 20:21:12,946 - INFO - 发现临时文件: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/backup_files/subfolder/sb.tmp
2025-06-09 20:21:12,946 - INFO - 发现临时文件: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_project/temp1.tmp
2025-06-09 20:21:12,947 - INFO - 发现临时文件: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_project/mp.tmp
2025-06-09 20:21:12,948 - INFO - 发现临时文件: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_project/data/backup.tmp


✅ 已删除文件: cache.tmp
🔍 搜索目录: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题
📋 找到 5 个临时文件:
  1. backup.tmp (路径: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/backup_files/backup.tmp)
  2. sb.tmp (路径: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/backup_files/subfolder/sb.tmp)
  3. temp1.tmp (路径: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_project/temp1.tmp)
  4. mp.tmp (路径: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_project/mp.tmp)
  5. backup.tmp (路径: /Users/huangwenxin/scripts/Code/python-learning-process-record/practices/python巩固语法练习题/my_project/data/backup.tmp)
👋 感谢使用临时文件管理器！


## 🎯 关键改进要点（纯os模块版本）：
1. 修复了删除逻辑Bug
```python
# ❌ 你的原代码问题：
os.remove(file_list[i][0])  # 删除文件
file_dict = dict(file_list)  # 但file_list还是原来的完整列表！

# ✅ 正确做法：
os.remove(target_path)
del self.temp_files[target_path]  # 立即从字典中移除
```
2. 精确的文件匹配
```python
# ❌ 你的原代码：
if ".tmp" in file:  # 会匹配"file.tmp.txt"

# ✅ 改进版本：
if filename.endswith('.tmp'):  # 只匹配真正的.tmp文件
```
3. 动态边界检查
```python
# ❌ 你的原代码：
if num < 1 or num > 11:  # 硬编码

# ✅ 改进版本：
if not (1 <= index <= len(self.temp_files)):  # 动态检查
```
4. 核心os模块函数使用
```python
# 路径操作 - 全部使用os模块
self.base_directory = os.path.abspath(base_directory)
full_path = os.path.join(self.base_directory, file_path)
directory = os.path.dirname(full_path)

# 目录创建
os.makedirs(directory, exist_ok=True)

# 文件遍历
for root, dirs, files in os.walk(self.base_directory):
    for filename in files:
        if filename.endswith('.tmp'):
            file_path = os.path.join(root, filename)

# 文件删除
os.remove(target_path)
```
5. 提供了两种实现方式
- 类版本：适合复杂项目，状态管理清晰
- 函数版本：更简单直接，适合学习os模块

✅ 成功修复了原来代码的所有问题：
1. 删除逻辑正确 - 删除文件后，列表实时更新（从8个变成7个）
2. 边界检查动态 - 会根据当前文件数量调整（1-8变成1-7）
3. 文件匹配精确 - 只找.tmp结尾的文件
4. 状态同步完美 - 每次操作后重新扫描显示最新状态
## 🎓 学习重点对比：
你的原版本 vs 改进版本：
| 问题       | 你的代码                    | 改进版本                          |
| :--------- | :-------------------------- | :-------------------------------- |
| 删除后状态 | file_dict = dict(file_list) | del self.temp_files[target_path]  |
| 文件匹配   | if ".tmp" in file           | if filename.endswith('.tmp')      |
| 边界检查   | if num < 1 or num > 11      | if not (1 <= index <= len(files)) |
| 路径处理   | 使用全局变量                | os.path.join(root, filename)      |
| 错误处理   | 基础try-except              | 具体的OSError处理                 |
## 🔧 os模块核心函数总结：
```python
# 你在这个项目中掌握的os模块函数：
os.path.abspath()     # 获取绝对路径
os.path.join()        # 路径拼接
os.path.dirname()     # 获取目录部分
os.path.exists()      # 检查路径存在
os.makedirs()         # 创建多级目录
os.walk()            # 递归遍历目录
os.remove()          # 删除文件
```

## 题目10：简易文件监控器 👀
**任务**: 创建一个简化版的文件监控器（就像之前的复杂题目的简化版）

**要求**:
1. 记录指定目录中所有文件的名称和大小
2. 等待几秒钟（可以手动添加/删除一些文件）
3. 再次扫描，比较前后差异
4. 报告发现的变化

**学习重点**: 综合运用前面学到的所有技能


In [ ]:
# 题目10：在这里写你的代码
import os
import time

def get_file_snapshot(directory):
    """获取目录中所有文件的快照"""
    # 你的代码写在这里：
    
    pass

def compare_snapshots(old_snapshot, new_snapshot):
    """比较两个快照的差异"""
    # 你的代码写在这里：
    
    pass

def simple_file_monitor(directory):
    """简易文件监控器主函数"""
    print(f"🎯 开始监控目录：{directory}")
    
    # 你的代码写在这里：
    
    pass

# 运行监控器
if os.path.exists("my_project"):
    simple_file_monitor("my_project")
else:
    print("❌ my_project目录不存在，请先完成前面的题目")


# 📚 学习总结与进阶指南

## 🎯 **通过这10个题目，你将掌握：**

### 🟢 **基础技能 (题目1-3)**
- ✅ 文件的创建、读取、写入
- ✅ os模块的基本路径操作
- ✅ 文件存在性检查和路径处理

### 🟡 **中级技能 (题目4-6)**  
- ✅ 目录结构的创建和管理
- ✅ 文件信息的获取（大小、时间等）
- ✅ 文件的复制和移动操作

### 🔴 **高级技能 (题目7-10)**
- ✅ 递归目录遍历 (`os.walk()`)
- ✅ JSON配置文件处理
- ✅ 文件过滤和批量操作
- ✅ 综合应用：简易文件监控器

---

## 📖 **核心函数速查表**

| 功能 | 函数 | 示例 |
|------|------|------|
| **文件读写** | `open()` | `with open("file.txt", "w") as f:` |
| **检查存在** | `os.path.exists()` | `os.path.exists("file.txt")` |
| **获取大小** | `os.path.getsize()` | `os.path.getsize("file.txt")` |
| **路径拼接** | `os.path.join()` | `os.path.join("dir", "file.txt")` |
| **列出目录** | `os.listdir()` | `os.listdir("directory")` |
| **创建目录** | `os.makedirs()` | `os.makedirs("path/to/dir")` |
| **递归遍历** | `os.walk()` | `for root, dirs, files in os.walk("dir"):` |
| **文件复制** | `shutil.copy()` | `shutil.copy("src", "dst")` |
| **文件移动** | `shutil.move()` | `shutil.move("src", "dst")` |
| **文件删除** | `os.remove()` | `os.remove("file.txt")` |

---

## 🚀 **完成练习后的进阶建议**

### **1. 学习现代Python路径操作**
```python
from pathlib import Path

# 现代化的路径操作
path = Path("my_project") / "data" / "file.txt"
if path.exists():
    print(path.stat().st_size)
```

### **2. 掌握上下文管理器**
```python
# 总是使用with语句
with open("file.txt", "r") as f:
    content = f.read()
# 文件自动关闭，即使出现异常
```

### **3. 学习异常处理**
```python
try:
    with open("file.txt", "r") as f:
        content = f.read()
except FileNotFoundError:
    print("文件不存在")
except PermissionError:
    print("没有权限访问文件")
```

### **4. 了解高级文件操作**
- 文件压缩和解压 (`zipfile`, `tarfile`)
- 临时文件操作 (`tempfile`)
- 文件锁定 (`fcntl`, `msvcrt`)
- 实时文件监控 (`watchdog`库)

---

## 💪 **学习建议**

1. **循序渐进** - 按题目顺序完成，不要跳跃
2. **多动手** - 每个函数都亲自尝试
3. **理解概念** - 不只是记住语法，要理解为什么这样做
4. **实际应用** - 想想这些技能在什么场景下有用
5. **查阅文档** - 遇到不懂的函数要查看官方文档

**记住：文件操作是编程的基础技能，掌握了它，你就能处理数据存储、配置管理、日志记录等许多实际问题！** 🎉

继续加油，相信完成这些练习后，你对文件操作会有全新的理解！ 💪
